[http://playground.tensorflow.org](http://playground.tensorflow.org)

In [1]:
import tensorflow as tf

w1=tf.random_normal(shape=(2,3),stddev=1,seed=1)
sess=tf.Session()
print sess.run(w1)
sess.close()

[[-0.81131822  1.48459876  0.06532937]
 [-2.4427042   0.0992484   0.59122431]]


In [2]:
w1=tf.Variable(tf.random_normal([2,3],stddev=1),name='w1')
w2=tf.Variable(tf.random_normal([2,2],stddev=1,dtype=tf.float64),name='w2')

#数据类型不一致报错
#w1.assign(w2)  

w3=tf.Variable(tf.random_normal([2,2],stddev=1),name='w3')
#维度不一致可以设置validate_shape=False
tf.assign(w1,w3,validate_shape=False)

<tf.Tensor 'Assign:0' shape=(2, 2) dtype=float32_ref>

In [25]:
import tensorflow as tf 
from numpy.random import RandomState
import numpy as np

batch_size=8


x=tf.placeholder(tf.float32,shape=(None,2),name='x-input')
y=tf.placeholder(tf.float32,shape=(None,1),name='y-input')

w1=tf.Variable(tf.random_normal([2,3],stddev=1))
a=tf.matmul(x,w1)

w2=tf.Variable(tf.random_normal([3,1],stddev=1))
prediction=tf.sigmoid(tf.matmul(a,w2))

tmp=tf.subtract(prediction,y)
tmp=tf.abs(tmp)
loss=tf.reduce_sum(tmp)
train_step=tf.train.GradientDescentOptimizer(0.01).minimize(loss)

rdm=RandomState(1)
dataset_size=1000
X=rdm.rand(dataset_size,2)
Y=[[int(x1+x2<1)] for (x1,x2) in X]


with tf.Session() as sess:
    init=tf.global_variables_initializer()
    sess.run(init)
    
    STEPS=50000
    for i in range(STEPS):
        start=(i*batch_size) % dataset_size
        end=min(start+batch_size,dataset_size)
        sess.run(train_step,feed_dict={x:X[start:end],y:Y[start:end]})
        
        if i<100 and i%10==0:
            tmp_loss=sess.run(loss/dataset_size,feed_dict={x:X,y:Y})
            
            print '训练轮数：%d，loss=%f' % (i,tmp_loss)
        
        if i%5000==0:
            tmp_loss=sess.run(loss/dataset_size,feed_dict={x:X,y:Y})
            print '训练轮数：%d，loss=%f' % (i,tmp_loss)
    
    

训练轮数：0，loss=0.485872
训练轮数：0，loss=0.485872
训练轮数：10，loss=0.467057
训练轮数：20，loss=0.450460
训练轮数：30，loss=0.438850
训练轮数：40，loss=0.431126
训练轮数：50，loss=0.420976
训练轮数：60，loss=0.417871
训练轮数：70，loss=0.415630
训练轮数：80，loss=0.410993
训练轮数：90，loss=0.407710
训练轮数：5000，loss=0.398780
训练轮数：10000，loss=0.396055
训练轮数：15000，loss=0.392307
训练轮数：20000，loss=0.390671
训练轮数：25000，loss=0.389889
训练轮数：30000，loss=0.389444
训练轮数：35000，loss=0.389157
训练轮数：40000，loss=0.388954
训练轮数：45000，loss=0.388803


In [28]:
import tensorflow as tf
v1=tf.constant([1,2,3,4])
v2=tf.constant([4,3,2,1])

with tf.Session() as sess:
    print tf.greater(v1,v2).eval()
    print tf.where(tf.greater(v1,v2),v1,v2).eval()

[False False  True  True]
[4 3 3 4]


In [35]:
import tensorflow as tf

from numpy.random import RandomState

batch_size=8

#两个输入节点
x=tf.placeholder(tf.float32,shape=(None,2),name='x-input')
y_=tf.placeholder(tf.float32,shape=(None,1),name='y-input')

w1=tf.Variable(tf.random_normal([2,1],stddev=1,seed=1))
y=tf.matmul(x,w1)

#定义预测多了和预测少了的成本
loss_less=10
loss_more=1

loss=tf.reduce_sum(tf.where(tf.greater(y,y_),(y-y_)*loss_more,(y_-y)*loss_less))
train_step=tf.train.AdamOptimizer(0.001).minimize(loss)

rdm=RandomState(1)
dataset_size=128
X=rdm.rand(dataset_size,2)
Y=[[x1+ x2+ rdm.rand()/10.0-0.05] for (x1,x2) in X]

with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())
    STEPS=5000
    for i in range(STEPS):
        start=(i* batch_size)% dataset_size
        end=min(start+batch_size,dataset_size)
        sess.run(train_step,feed_dict={x:X[start:end],y_:Y[start:end]})
        
    print sess.run(w1)

[[ 1.01934695]
 [ 1.04280889]]


In [29]:
#指数衰减（decay）的学习率
global_step=tf.Variable(0)

learn_rate=tf.train.exponential_decay(0.1,global_step,100,0.96,staircase=True)
train_step=tf.train.GradientDescentOptimizer(learn_rate).minimize(...loss...,global_step=global_step)

0.5769778464775016

In [ ]:
#正则化
loss=tf.reduce_mean(tf.square(y_-y)) + tf.contrib.layers.l2_regularizer(lambda_)(w)